<a href="https://colab.research.google.com/github/Andicleomj/Machine-Learning/blob/main/Week10/Classification%20model%3A%20MLP%20Classification/Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Andi Cleopatra Maryam jamila
#### 1103213071
#### Classification model: MLP Classification


In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [27]:
# Memuat dataset Iris
iris = load_iris()
X = iris.data
y = iris.target

In [28]:
# Menstandarisasi fitur
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [29]:
# Membagi dataset menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [30]:
# Mengonversi ke tensor
X_train_tensor = torch.Tensor(X_train)
X_test_tensor = torch.Tensor(X_test)
y_train_tensor = torch.Tensor(y_train).long()
y_test_tensor = torch.Tensor(y_test).long()

In [31]:
# Membuat DataLoader untuk training dan testing
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

In [32]:
# Fungsi untuk membuat DataLoader
def create_dataloader(batch_size):
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

In [33]:
# Mendefinisikan Model MLP
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_layers, neurons_per_layer, activation_fn):
        super(MLP, self).__init__()
        layers = []
        current_input = input_dim

        # Membuat hidden layers
        for _ in range(hidden_layers):
            layers.append(nn.Linear(current_input, neurons_per_layer))
            if activation_fn == 'ReLU':
                layers.append(nn.ReLU())
            elif activation_fn == 'Sigmoid':
                layers.append(nn.Sigmoid())
            elif activation_fn == 'Tanh':
                layers.append(nn.Tanh())
            current_input = neurons_per_layer

        # Output layer
        layers.append(nn.Linear(current_input, 3))  # 3 kelas pada dataset Iris
        layers.append(nn.Softmax(dim=1))  # Softmax untuk klasifikasi multi-kelas

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


In [34]:
# Fungsi Pelatihan
def train_model(model, train_loader, learning_rate, epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    model.train()
    train_loss = []
    for epoch in range(epochs):
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        train_loss.append(epoch_loss / len(train_loader))
        print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss / len(train_loader)}')

    return train_loss


In [35]:
# Contoh eksperimen
hidden_layers = 2
neurons_per_layer = 16
activation_fn = 'ReLU'
learning_rate = 0.01
epochs = 50
batch_size = 64

train_loader, test_loader = create_dataloader(batch_size)
model = MLP(input_dim=4, hidden_layers=hidden_layers, neurons_per_layer=neurons_per_layer, activation_fn=activation_fn)

# Melatih model
train_loss = train_model(model, train_loader, learning_rate, epochs)


Epoch 1/50, Loss: 1.0898390412330627
Epoch 2/50, Loss: 1.061174988746643
Epoch 3/50, Loss: 1.0321409106254578
Epoch 4/50, Loss: 1.0003392398357391
Epoch 5/50, Loss: 0.9642347693443298
Epoch 6/50, Loss: 0.9184033870697021
Epoch 7/50, Loss: 0.8969387710094452
Epoch 8/50, Loss: 0.8513369560241699
Epoch 9/50, Loss: 0.8078986406326294
Epoch 10/50, Loss: 0.7815707325935364
Epoch 11/50, Loss: 0.7582186162471771
Epoch 12/50, Loss: 0.736363559961319
Epoch 13/50, Loss: 0.7200369238853455
Epoch 14/50, Loss: 0.7127375602722168
Epoch 15/50, Loss: 0.6880621314048767
Epoch 16/50, Loss: 0.6804692447185516
Epoch 17/50, Loss: 0.6741258203983307
Epoch 18/50, Loss: 0.6526428461074829
Epoch 19/50, Loss: 0.6499209105968475
Epoch 20/50, Loss: 0.6417641937732697
Epoch 21/50, Loss: 0.6269614100456238
Epoch 22/50, Loss: 0.6249173879623413
Epoch 23/50, Loss: 0.6199700832366943
Epoch 24/50, Loss: 0.6097123026847839
Epoch 25/50, Loss: 0.6029579341411591
Epoch 26/50, Loss: 0.5973392426967621
Epoch 27/50, Loss: 0.59

In [36]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    accuracy = correct / total
    return accuracy

# Evaluasi model
accuracy = evaluate_model(model, test_loader)
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 100.00%
